-> Google Sheets (Google Planilhas): É um software de planilhas que faz parte
do pacote do Google Workspace. Diferente do Excel tradicional, ele não "mora"
apenas no seu computador; ele vive nos servidores do Google.

-> Diferenciais Técnicos para Automação: 

-> 1. Base em Nuvem (Saas): Por ser um Software as a Service, o acesso aos dados
é feito via URL. Isso facilita muito a colaboração em tempo real, pois
varias pessoas (ou robôs) podem editar o arquivo ao mesmo tempo.

-> 2. API do Google Sheets: Para nós, programadores, essa é a parte mais 
importante. O Google fornece uma API (Interface de Programação de Aplicações)
que permite ao Python ler, escrever e formatar células sem que você precise
abrir o navegador manualmente.

-> 3. Integração Nativa: Ele se conecta facilmente com outras ferramentas
(Google Forms, Drive, Gmail), o que permite criar fluxos onde, por exemplo,
um cliente preenche um formulário e o seu script Python detecta a nova linha
e já envia o whatsapp automaticamente.

-> API: Tem como objetivo permitir a comunicação entre diferentes programas e ferramentas.


                        Etapas de Configuração da API Google Sheet

Passo 1: Criar o projeto no google cloud

    -> Acesse o site https://console.cloud.google.com/apis/dashboard?project=infra-lodge-485214-e8 ou pesquise por google developer console

    -> Clique no seletor de projetos no topo da página e selecione "Novo Projeto".

    -> Dê um nome ao projeto e clique em criar.

Passo 2: Ativar a API do Google Sheets e do Google Drive

    -> No menu lateral, vá em APIs e Serviços > Biblioteca.

    -> Pesquise por "Google Sheets API" e clique nela.

    -> clique no botão ativar
    
    -> Faça o mesmo com a api do google drive.(pesquise e ative a api)

Passo 3: Configurar a tela de consentimento

    -> No menu lateral, clique em Tela de permissão OAuth (ou Google Auth Platform).

    -> Escolha o tipo de usuário EXTERNO.

    -> Informações do App: Preencha o nome do app e seu e-mail de suporte.

Passo 4: Definindo o Escopo:

    -> no menu lateral clique em "Acesso a dados" e depois em "adicionar ou remover escopos"

    -> Procure por "google sheet api" e marque-o

    -> Salve as alterações no fim da página.

Passo 5: Definindo Usuários de Testes:

    -> No menu lateral clique em "Público-alvo" 

    -> clique em "Add users" e coloque o seu email. Dessa forma você conseguira
    logar no sistema com o seu email.

Passo 5: Criar Credenciais no computador:

    -> No menu lateral vá em "clientes" ou "credenciais"

    -> Clique em  CRIAR CREDENCIAIS > ID do cliente OAuth.

    -> Em "Tipo de aplicativo", escolha App de computador.

    -> Dê um nome e clique em Criar.

    -> Na lista que aparecer ou no lapis de edição, clique no ícone de Download (Seta para baixo) para baixar o arquivo JSON.


Passo 6: Preparando o ambiente python

    -> Coloque o arquivo JSON na mesma pasta do seu script .py ou .ipynb

    -> Renomeie o arquivo para algo simples, como credencial.json.

    -> Instale as bibliotecas necessárias no seu terminal


Observação: Caso o código não funcione no navegador do google chrome (como erro 400 que indica problemas no link do escopo), tente utilizar outro navegador.
Eu utilizei o navegador avast secure browser.

Biblioteca que deve ser instalada: pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib



Copiando o código de amostra que configura o python para acessar o servidor do google

pesquisa: Google  sheets api python

site: python quick starts (https://developers.google.com/workspace/sheets/api/quickstart/python)

In [ ]:
# Serve para o python conversar com o sistema de arquivos do computador
# (Windows/Mac/Linux). Ele será usado para verificar se o arquivo token.json
# já existe na nossa pasta. Se existir ele pulará a etapa de lohgin
import os.path

# Gerencia o envio de pedidos de atualização de segurança. Quando a chave
# token.json expira, essa biblioteca envia um pedido automático ao Google
# para renova-la sem que você precise logar de novo.
from google.auth.transport.requests import Request

# É o formato padrão que o Google usa para entender as credenciais.
# Ela lê o conteudo do arquivo token.json e o transforma em um objeto
# que o python consegue usar para provar quem você é.
from google.oauth2.credentials import Credentials

# É o "motor" que cria o processo de login para aplicativos instalados no
# computador. Ele pega o credencialcliente1.json e cria aquele link que
# abre o navegador para clicarmos em permitir.
from google_auth_oauthlib.flow import InstalledAppFlow

# É a biblioteca "construtura". Ela serve para conectar o Python a qualquer
# serviço do Google (Sheets, Drive, YouTube, Gmail). Usamos ela para dizer:
# "Quero construir uma conexão com a API de sheets, na versão v4". É ela
# quem cria o objeto service.
from googleapiclient.discovery import build

# Biblioteca de monitoramento de erros especificos da internet (HTTP).
# Se a internet cair, ou se você digitar o ID da planilha errado, ele
# captura esse erro e te avisa (print(err)) em vez de simplesmente travar
# o programa com um erro genérico.
from googleapiclient.errors import HttpError

# Define o nivel de acesso que podemos ter na planilha.
# Esses niveis são definidos adicionando ".alguma coisa",
# por exemplo: .readonly define que o usuário pode apenas
# ler o arquivo.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# Define o id da planilha que iremos acessar no google sheets.
SAMPLE_SPREADSHEET_ID = "1P4AcjzVeV9GuC2_NsWNl9LpKJqDb7QjdV6xXrM5ct08"

# Define o intervalo de celulas que iremos manipular na planilha
SAMPLE_RANGE_NAME = "Página1!A1:E16"


def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  # Possui como valor inicial o None, pois, não temos credenciais (arquivo
  # token.json) no primeiro acesso.
  creds = None
  
  if os.path.exists("token.json"):
    # O código verifica se ja fizemos login antes. Se o arquivo token.json
    # existir, ele carrega as nossas credenciais salvas para você não precisar
    # logar toda vez.
    # token,js: Arquivo que contém a chave de acesso ao sistema. Ela é criada
    # na pasta após realizarmos o primeiro acesso ao sistema. Até onde entendi
    # o motivo para passarmos o token.json e o scope como argumento da função
    # from_authorized_user_file é por que um representa o nosso acesso ao
    # sistema e o outro representa o que podemos fazer dentro do sistema
    # (permissões).
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

  if not creds or not creds.valid:
    
    # Se a chave (token.json) expirou, mas você ainda tem o codigo de renovação,
    # Python tenta renovar o acesso automaticamente em segundo plano.
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      
      # Se você nunca logou, ele usa o seu arquivo JSON (credencialcliente1 baixado na tela de credenciais do google cloud) para abrir o navegador
      # na porta 8080 e pedir a sua autorização manual.
      flow = InstalledAppFlow.from_client_secrets_file(
          "credencialcliente1.json", SCOPES
      )
      creds = flow.run_local_server(port=8080, prompt='consent')
      
    with open("token.json", "w") as token:
      
      # Após o login bem sucedido, ele cria o token.json para que, na proxima 
      # vez, o login seja automático.
      token.write(creds.to_json())

  try:
    
    # Cria a conexão oficial com o serviço do Google Sheet (versão 4). O
    # objeto sheet vira o nosso "controle remoto".
    # Serve para ler informações do google sheets
    service = build("sheets", "v4", credentials=creds)
    sheet = service.spreadsheets()
    
    # O comando .get vai até a nuvem (Google Sheets) e traz o dado do intervalo
    # que definimos em SAMPLE_RANGE_NAME da planilha que estamos manipulando
    # (especificada na variável SAMPLE_SPREADSHEET_ID através do seu id ).
    # O execute serve para enviar a nossa requisição ao Google Drive.
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    
    print("Lista de valores da planilha")
    # Ira armazenar o dicionário de valores retornado pelo get que fizemos ácima
    valores = result['values']
    print(valores)
    
    for linhas in valores:
      # Ira percorrer os valores do dicionário e imprimi-lo em formato de lista.
      print(linhas)
      
    # Adicionar uma informação
    
    # Lista de valores que queremos adicionar na planilha 
    valores_adicionar = [["Março", "R$ 127.300,15", "R$ 15.000,00"]]
    
    # Usa o comando .update para gravar os valores da lista valores_adicionar
    # na celula A16 da planilha. O USER_ENTERED é fundamental para que o Google
    # entenda o "R$" como moeda e não apenas como texto. Diferença entre RAW e
    # USER_ENTERED:
    # RAW: Indica que o dado informado deve ser tratado exclusivamente como texto,
    # por exemplo, 1 + 2 no raw não é 3 e sim '1+2' literalmente.
    # USER_ENTERED: Indica que o dado informado deve ser armazenado da forma
    # que o usuário deve interpreta-lo, por exemplo, 1 + 2 em USER_ENTERED
    # é 3, por que o usuário entende isso como uma fórmula de soma.
    # body: Contém a lista de valores que serão adicionados na planilha.
    result = (
        sheet.values()
        .update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="A16", valueInputOption="USER_ENTERED", body = {'values': valores_adicionar } )
        .execute()
    )
    
    # Criando uma coluna nova: Vamos criar uma coluna que irá conter a comissão
    # de cada vendedor. A comissão será 10% de cada venda.
    
    # Cria uma lista que será a nossa nova coluna, já começando com o titulo
    # (cabeçalho).
    comissao_vendedor = [
      
      ["comissão vendedor"],
         
    ]
    
    print("Valores das vendas")
    # Percorre cada linha da planilha
    for i, linha in enumerate(valores):
      
      # Verifica se o indice da linha percorrida é maior que 0
      # (linha que contém apenas o cabeçalho da planilha)
      if i > 0:
        
        # Se essa condição for verdadeira, vamos armazenar os valores 
        # da segunda coluna na variável vendas
        vendas = linha[1]
        
        # Primeiro iremos substituir o "R$" e o "." por um espaço em branco
        vendas = vendas.replace("R$","").replace(".", "")
        
        # Após isso, iremos substituir as "," por pontos e converter os valores
        # para float.
        vendas = float(vendas.replace(",","."))
        
        # Ira somar os valores por 10%
        imposto = vendas * 0.1
        
        # Ira adicionar os valores na lista de valores da nossa nova coluna
        comissao_vendedor.append([imposto])
        # Ira imprimir as vendas convertidas em float
        print(vendas)
    
    # Ira adicionar a nossa nova coluna e as suas comissões na celula D1
    result = (
        sheet.values()
        .update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="D1", valueInputOption="USER_ENTERED", body = {'values': comissao_vendedor } )
        .execute()
    )
    
    # Ira criar uma nova lista que se tornara uma coluna na nossa planilha.
    nome_vendedor = [["Vendedor"], ["Ricardo"], ["Marcos"], ["Lenadro"], ["Jean"], ["Karina"], ["Geovana"], ["Erika"], ["Luiz"], ["Priscila"],
                     ["carlos"], ["Thiago"], ["Felipe"], ["Raul"], ["Larissa"], ["Otavio"]]
    
    # Ira incluir os valores da lista nome_vendedor na célula E1
    result = (
        sheet.values()
        .update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="E1", valueInputOption="USER_ENTERED", body = {'values': nome_vendedor } )
        .execute()
    )
  except HttpError as err:
    
    # Se algo der errado (como falta de internet ou ID errado), o código não 
    # "quebra" feio, ele apenas imprime o erro para você saber o que aconteceu.
    print(err)


if __name__ == "__main__":
    main()

Lista de valores da planilha
[['Mês', 'Vendas', 'Imposto', 'comissão vendedor'], ['janeiro', 'R$ 50.668,00', 'R$ 5.066,80', 'R$ 5.066,80'], ['fevereiro', 'R$ 98.475,00', 'R$ 9.847,50', 'R$ 9.847,50'], ['março', 'R$ 92.030,00', 'R$ 9.203,00', 'R$ 9.203,00'], ['abril', 'R$ 63.546,00', 'R$ 6.354,60', 'R$ 6.354,60'], ['maio', 'R$ 24.496,00', 'R$ 2.449,60', 'R$ 2.449,60'], ['junho', 'R$ 69.997,00', 'R$ 6.999,70', 'R$ 6.999,70'], ['julho', 'R$ 35.242,00', 'R$ 3.524,20', 'R$ 3.524,20'], ['agosto', 'R$ 73.105,00', 'R$ 7.310,50', 'R$ 7.310,50'], ['setembro', 'R$ 57.397,00', 'R$ 5.739,70', 'R$ 5.739,70'], ['outubro', 'R$ 30.293,00', 'R$ 3.029,30', 'R$ 3.029,30'], ['novembro', 'R$ 17.345,00', 'R$ 1.734,50', 'R$ 1.734,50'], ['dezembro', 'R$ 127.300,15', 'R$ 12.730,02', 'R$ 12.730,02'], ['janeiro', 'R$ 100.000,00', 'R$ 10.000,00', 'R$ 10.000,00'], ['fevereiro', 'R$ 500.000,00', 'R$ 100.000,00', 'R$ 50.000,00'], ['Março', 'R$127.300,15', 'R$15.000,00', 'R$ 12.730,02']]
['Mês', 'Vendas', 'Imposto', '